<a href="https://colab.research.google.com/github/aledelorbe/pt2Codigos/blob/main/SegundasActividades/Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Cargar librerias

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
from sklearn.cluster import KMeans

### Cargar los dataFrame

In [24]:
nuevaMortalidadLimpia = '/content/drive/MyDrive/Pt2/Primeras Actividades/0 Bases de Datos/Nueva Mortalidad Limpia/'

df2010 = pd.read_csv(nuevaMortalidadLimpia + '2010.csv')
df2011 = pd.read_csv(nuevaMortalidadLimpia + '2011.csv')
df2012 = pd.read_csv(nuevaMortalidadLimpia + '2012.csv')
df2013 = pd.read_csv(nuevaMortalidadLimpia + '2013.csv')
df2014 = pd.read_csv(nuevaMortalidadLimpia + '2014.csv')
df2015 = pd.read_csv(nuevaMortalidadLimpia + '2015.csv')
df2016 = pd.read_csv(nuevaMortalidadLimpia + '2016.csv')
df2017 = pd.read_csv(nuevaMortalidadLimpia + '2017.csv')
df2018 = pd.read_csv(nuevaMortalidadLimpia + '2018.csv')
df2019 = pd.read_csv(nuevaMortalidadLimpia + '2019.csv')

In [25]:
df2010

,ent_resid_nm,Organ,desc_ocupacion,desc_escolaridad
0,Aguascalientes,Pancreas,"Trabajadores en actividades agrícolas, ganader...",Sin escolaridad
1,Aguascalientes,Bladder,"Trabajadores en actividades agrícolas, ganader...",Sin escolaridad
2,Aguascalientes,Prostate,"Trabajadores en actividades agrícolas, ganader...",No especificada
3,Aguascalientes,Lungs,"Trabajadores en actividades agrícolas, ganader...",Preescolar
4,Aguascalientes,Lungs,"Trabajadores en actividades agrícolas, ganader...",Sin escolaridad
...,...,...,...,...
70138,Zacatecas,Brain,Trabajadores en actividades elementales y de a...,Sin escolaridad
70139,Zacatecas,Cervix Uteri,Trabajadores en actividades elementales y de a...,Sin escolaridad
70140,Zacatecas,Brain,Trabajadores en actividades elementales y de a...,Sin escolaridad
70141,Coahuila de Zaragoza,Male Genitales,Trabajadores en actividades elementales y de a...,Preescolar




### Transformaciones

#### Educación

In [26]:
# Esta columna requiere una transformacion distinta a las demas columnas, pues sus valores distintos deben ser transformados a numeros y cada uno de
# estos numeros pueden ser mayores o menores que otros por la jerarquia, es decir, una persona con nivel de eduacion profesional debe tener un numero mas
# grande que una persona con nivel de eduacion secundaria.

# Definicion de la funcion que en cada fragmento codificara a la variable 'desc_escolaridad'
def codificadorOrdinalFragmentos(codificadorOrdinalX, df20xx, nombreDf):
  # Ajustamos el codificador con la variable desc_escolaridad y la transformamos
  codificadorOrdinalX.fit(df20xx[["desc_escolaridad"]])
  df20xx["escolaridad_codificada"] = codificadorOrdinalX.transform(df20xx[["desc_escolaridad"]])

  # Imprimir resultado
  titulo = f"DataFrame {nombreDf}".center(100, '-')
  print(titulo)
  parejas_distintas = df20xx[['desc_escolaridad', 'escolaridad_codificada']].drop_duplicates().sort_values(by='escolaridad_codificada', ascending=True)
  print(parejas_distintas)

  return df20xx

# Crear la jerarquia (de menor a mayor jerarquia)
valoresDistintosEducacion = [
    'No especificada ',
    'No aplica a menores de 3 años',
    'Sin escolaridad',
    'Preescolar',
    'Primaria completa',
    'Secundaria incompleta',
    'Secundaria completa',
    'Bachillerato o preparatoria',
    'Profesional',
    'Posgrado',
]

# Creamos el codificador indicandole el orden de la variables
codificadorOrdinal = OrdinalEncoder(categories=[valoresDistintosEducacion])

# Mandar a llamar la funcion
df2010 = codificadorOrdinalFragmentos(codificadorOrdinal, df2010, '2010')
df2011 = codificadorOrdinalFragmentos(codificadorOrdinal, df2011, '2011')
df2012 = codificadorOrdinalFragmentos(codificadorOrdinal, df2012, '2012')
df2013 = codificadorOrdinalFragmentos(codificadorOrdinal, df2013, '2013')
df2014 = codificadorOrdinalFragmentos(codificadorOrdinal, df2014, '2014')
df2015 = codificadorOrdinalFragmentos(codificadorOrdinal, df2015, '2015')
df2016 = codificadorOrdinalFragmentos(codificadorOrdinal, df2016, '2016')
df2017 = codificadorOrdinalFragmentos(codificadorOrdinal, df2017, '2017')
df2018 = codificadorOrdinalFragmentos(codificadorOrdinal, df2018, '2018')
df2019 = codificadorOrdinalFragmentos(codificadorOrdinal, df2019, '2019')

-------------------------------------------DataFrame 2010-------------------------------------------
                    desc_escolaridad  escolaridad_codificada
2                   No especificada                      0.0
49952  No aplica a menores de 3 años                     1.0
0                    Sin escolaridad                     2.0
3                         Preescolar                     3.0
11                 Primaria completa                     4.0
28             Secundaria incompleta                     5.0
114              Secundaria completa                     6.0
101      Bachillerato o preparatoria                     7.0
160                      Profesional                     8.0
-------------------------------------------DataFrame 2011-------------------------------------------
                    desc_escolaridad  escolaridad_codificada
57                  No especificada                      0.0
41068  No aplica a menores de 3 años                     1.0
20   

#### Empleo

In [27]:
# Definicion de la funcion que dumizara distintas variables en cada fragmento
def duminizador(df20xx, nombreDf, nombreVariable):
  # Dumizar variable
  dummies = pd.get_dummies(df20xx[nombreVariable], drop_first = True)
  # drop_first = True -> es para que nos agregue solo k-1 categorias distintas y no se agregue la categoria k la cual solo provoca redundancia
  df20xx = pd.concat([df20xx, dummies], axis = 1) # Añadimos las variables binarias al DataFrame

  # Solicitud de las categorias que se encuentran en la variable: 'desc_ocupacion'
  nombresCategoriasX = dummies.columns.tolist()

  # Imprimir resultado
  titulo = f"DataFrame {nombreDf}".center(100, '-')
  print(titulo)
  parejas_distintas = df20xx[[nombreVariable] + nombresCategoriasX].drop_duplicates().reset_index(drop=True)
  print(parejas_distintas.to_string())

  return df20xx

# Mandar a llamar la funcion
df2010 = duminizador(df2010, '2010', 'desc_ocupacion')
df2011 = duminizador(df2011, '2011', 'desc_ocupacion')
df2012 = duminizador(df2012, '2012', 'desc_ocupacion')
df2013 = duminizador(df2013, '2013', 'desc_ocupacion')
df2014 = duminizador(df2014, '2014', 'desc_ocupacion')
df2015 = duminizador(df2015, '2015', 'desc_ocupacion')
df2016 = duminizador(df2016, '2016', 'desc_ocupacion')
df2017 = duminizador(df2017, '2017', 'desc_ocupacion')
df2018 = duminizador(df2018, '2018', 'desc_ocupacion')
df2019 = duminizador(df2019, '2019', 'desc_ocupacion')

-------------------------------------------DataFrame 2010-------------------------------------------
                                                                              desc_ocupacion  Funcionarios, directores y jefes  Insuficientemente especificada  No aplica a menores de 5 años  No especificada  No trabaja  Operadores de maquinaria industrial, ensambladores, choferes y conductores de transporte  Profesionistas y técnicos  Trabajadores artesanales  Trabajadores auxiliares en actividades administrativas  Trabajadores en actividades agrícolas, ganaderas, forestales, caza y pesca  Trabajadores en actividades elementales y de apoyo  Trabajadores en servicios personales y vigilancia
0                 Trabajadores en actividades agrícolas, ganaderas, forestales, caza y pesca                                 0                               0                              0                0           0                                                                                    

In [28]:
df2010 = df2010.drop(columns=['desc_ocupacion']) # Eliminamos la vairable original 'nombreDescripcion'
df2010

,ent_resid_nm,Organ,desc_escolaridad,escolaridad_codificada,"Funcionarios, directores y jefes",Insuficientemente especificada,No aplica a menores de 5 años,No especificada,No trabaja,"Operadores de maquinaria industrial, ensambladores, choferes y conductores de transporte",Profesionistas y técnicos,Trabajadores artesanales,Trabajadores auxiliares en actividades administrativas,"Trabajadores en actividades agrícolas, ganaderas, forestales, caza y pesca",Trabajadores en actividades elementales y de apoyo,Trabajadores en servicios personales y vigilancia
0,Aguascalientes,Pancreas,Sin escolaridad,2.0,0,0,0,0,0,0,0,0,0,1,0,0
1,Aguascalientes,Bladder,Sin escolaridad,2.0,0,0,0,0,0,0,0,0,0,1,0,0
2,Aguascalientes,Prostate,No especificada,0.0,0,0,0,0,0,0,0,0,0,1,0,0
3,Aguascalientes,Lungs,Preescolar,3.0,0,0,0,0,0,0,0,0,0,1,0,0
4,Aguascalientes,Lungs,Sin escolaridad,2.0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70138,Zacatecas,Brain,Sin escolaridad,2.0,0,0,0,0,0,0,0,0,0,0,1,0
70139,Zacatecas,Cervix Uteri,Sin escolaridad,2.0,0,0,0,0,0,0,0,0,0,0,1,0
70140,Zacatecas,Brain,Sin escolaridad,2.0,0,0,0,0,0,0,0,0,0,0,1,0
70141,Coahuila de Zaragoza,Male Genitales,Preescolar,3.0,0,0,0,0,0,0,0,0,0,0,1,0


#### Cancer

In [29]:
# Mandar a llamar la funcion dumizadora aplicandola a la variable cancer
df2010 = duminizador(df2010, '2010', 'Organ')
df2011 = duminizador(df2011, '2011', 'Organ')
df2012 = duminizador(df2012, '2012', 'Organ')
df2013 = duminizador(df2013, '2013', 'Organ')
df2014 = duminizador(df2014, '2014', 'Organ')
df2015 = duminizador(df2015, '2015', 'Organ')
df2016 = duminizador(df2016, '2016', 'Organ')
df2017 = duminizador(df2017, '2017', 'Organ')
df2018 = duminizador(df2018, '2018', 'Organ')
df2019 = duminizador(df2019, '2019', 'Organ')

-------------------------------------------DataFrame 2010-------------------------------------------
                      Organ  Bone  Brain  Breast  Cervix Uteri  Colon and Rectum  Esophagus  Eyes  Female Genitals  Gallbladder  Heart  Hodgkin's Lymphoma  Kidney  Larynx  Leukemia  Liver   Lungs  Lymphoma  Male Genitales  Melanoma  Mesothelioma  Mylenoma  Nervous  Oral Cavity and Pharynx  Outter Respiratory  Ovary  Pancreas  Prostate  Skin non Melanoma   Soft Tissue  Stomach  Testis  Thyroid  Trachea  Ureter  Urinary Tract Other
0                  Pancreas     0      0       0             0                 0          0     0                0            0      0                   0       0       0         0       0      0         0               0         0             0         0        0                        0                   0      0         1         0                   0            0        0       0        0        0       0                    0
1                   Bladder    

#### Entidad Federativa

In [30]:
# Mandar a llamar la funcion dumizadora aplicandola a la variable 'Entidad Federativa'
df2010 = duminizador(df2010, '2010', 'ent_resid_nm')
df2011 = duminizador(df2011, '2011', 'ent_resid_nm')
df2012 = duminizador(df2012, '2012', 'ent_resid_nm')
df2013 = duminizador(df2013, '2013', 'ent_resid_nm')
df2014 = duminizador(df2014, '2014', 'ent_resid_nm')
df2015 = duminizador(df2015, '2015', 'ent_resid_nm')
df2016 = duminizador(df2016, '2016', 'ent_resid_nm')
df2017 = duminizador(df2017, '2017', 'ent_resid_nm')
df2018 = duminizador(df2018, '2018', 'ent_resid_nm')
df2019 = duminizador(df2019, '2019', 'ent_resid_nm')

-------------------------------------------DataFrame 2010-------------------------------------------
                      Organ  Bone  Bone  Brain  Brain  Breast  Breast  Cervix Uteri  Cervix Uteri  Colon and Rectum  Colon and Rectum  Esophagus  Esophagus  Eyes  Eyes  Female Genitals  Female Genitals  Gallbladder  Gallbladder  Heart  Heart  Hodgkin's Lymphoma  Hodgkin's Lymphoma  Kidney  Kidney  Larynx  Larynx  Leukemia  Leukemia  Liver   Liver   Lungs  Lungs  Lymphoma  Lymphoma  Male Genitales  Male Genitales  Melanoma  Melanoma  Mesothelioma  Mesothelioma  Mylenoma  Mylenoma  Nervous  Nervous  Oral Cavity and Pharynx  Oral Cavity and Pharynx  Outter Respiratory  Outter Respiratory  Ovary  Ovary  Pancreas  Pancreas  Prostate  Prostate  Skin non Melanoma   Skin non Melanoma   Soft Tissue  Soft Tissue  Stomach  Stomach  Testis  Testis  Thyroid  Thyroid  Trachea  Trachea  Ureter  Ureter  Urinary Tract Other  Urinary Tract Other
0                  Pancreas     0     0      0      0      